
# Diagnose und Klassifikation von Diabetes mellitus mithilfe maschinellen Lernens

Hier finden Sie den gesamten Code, den der Autor für die Analyse verwendet hat.

Auch einzusehen unter: https://github.com/NicoPiel/LiteraturSeminar

In [1]:
# Setup

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

%matplotlib inline
sns.set_theme()

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Daten importieren
data = pd.read_csv('data/diabetes.csv')

# Nullen durch NaN ersetzen
data["Glucose"] = data["Glucose"].replace(0, np.nan)
data["BloodPressure"] = data["BloodPressure"].replace(0, np.nan)
data["SkinThickness"] = data["SkinThickness"].replace(0, np.nan)
data["Insulin"] = data["Insulin"].replace(0, np.nan)
data["BMI"] = data["BMI"].replace(0, np.nan)
data["DiabetesPedigreeFunction"] = data["DiabetesPedigreeFunction"].replace(0, np.nan)
data["Age"] = data["Age"].replace(0, np.nan)

# NaN Werte imputieren
imputer = SimpleImputer()

data_imputed = pd.DataFrame(imputer.fit_transform(data))
data_imputed.columns = data.columns

# Vorbereitung

Als erstes möchten wir uns ein wenig Übersicht verschaffen, also Data Exploration betreiben.

In [ ]:
# Spalten

data.columns

In [ ]:
# Übersicht verschaffen

data.head()

In [ ]:
data.describe()

Die einzelnen Spalten haben folgende Bedeutung:

- **Pregnancies:** Anzahl der bisherigen (erfolgreichen und gescheiterten) Schwangerschaften
- **Glucose:** 75g-2-h-oGTT in $\frac{\text{mg}}{\text{dl}}$. Glukosewert nach 75g Kohlenhydraten und zwei Stunden Wartezeit
- **BloodPressure:** Diastolischer Blutdruck in $\text{mm} \cdot \text{Hg}$
- **SkinThickness:** Dicke der Trizepshautfalte in $\text{mm}$
- **Insulin:** Seruminsulin nach 2 Stunden in $\frac{\text{µU}}{\text{ml}}$
- **BMI:** Body-Mass-Index in $\frac{\text{kg}}{\text{m}^2}$
- **DiabetesPedigreeFunction:** Stammbaumfunktion des Patienten
- **Age:** Alter in Jahren
- **Outcome:** Klassifikation. 1 = Diabetes, 0 = kein Diabetes

In [ ]:
# Keine sichtbare Beziehung zwischen

sns.pairplot(data = data, hue="Outcome")

In [ ]:
# Pearson-Koeffizienten für alle Spalten

print("Pearson-Koeffizient > 0.5")

for col in data_imputed:
    for col2 in data_imputed:
        if (col != col2):
            arr1 = data_imputed[col].to_numpy()
            arr2 = data_imputed[col2].to_numpy()
        
            pearson = stats.pearsonr(arr1, arr2)
            
            if (pearson[0] > 0.4):
                print(f"{col} -> {col2}: {pearson[0]}")

Aus den Pearson-Koeffizienten lässt sich schließen, dass die klinischen Werte keinen direkten Einfluss aufeinander haben. 

Die Anzahl der Schwangerschaften scheint mit dem Alter, die Dicke der Trizeps-Hautfalte mit dem BMI zu korrelieren.
Überraschenderweise scheint der 75g-2h-oGGT nur mittelmäßig mit dem Ergebnis zusammenzuhängen.

# Das Modell

In [ ]:
X = data_imputed.drop('Outcome', inplace=False)
y = data_imputed['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

# early_stopping = EarlyStopping()

network = keras.Sequential(
    layers.Dense(512, activation='relu', input_shape=[11]),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(1),
)